# Federal Account Data Collection from USASpending.gov

This notebook implements a comprehensive three-stage data collection process for federal account information:

## 🎯 **Collection Strategy**

1. **Stage 1**: Collect all budget functions and subfunctions from USASpending.gov
2. **Stage 2**: Create hierarchical mapping between functions and subfunctions  
3. **Stage 3**: Collect detailed federal account data for each function-subfunction pair across all fiscal years and quarters

## 📊 **Expected Output**

- **Budget Functions**: ~20 high-level categories (e.g., National Defense, Social Security)
- **Budget Subfunctions**: ~70 detailed breakdowns within functions
- **Federal Accounts**: Thousands of specific accounts with obligation amounts
- **Time Coverage**: FY 2017-2024 (quarterly data)

---

In [ ]:
import requests
import pandas as pd
import time
import os

## 📦 Required Libraries

Essential imports for the federal account data collection:
- **`requests`**: HTTP API calls to USASpending.gov endpoints
- **`pandas`**: Data manipulation and CSV file operations  
- **`time`**: Delays for API rate limiting
- **`os`**: File system operations and directory management

## 🔗 Google Drive Mount

Establishes connection to Google Drive for data storage:
- **Mount Point**: `/content/drive/` provides access to your Google Drive
- **Storage Location**: Data will be saved to `My Drive/USASpendingResults/`
- **Persistence**: Ensures collected data is saved permanently to your Google Drive

*Note: This is specific to Google Colab environment*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 1. Fetch all budget functions
functions_url = "https://api.usaspending.gov/api/v2/budget_functions/list_budget_functions/"
response = requests.get(functions_url)
response.raise_for_status()
data = response.json()

df_budget_functions = pd.json_normalize(data['results'])
df_budget_functions



,budget_function_code,budget_function_title
0,750,Administration of Justice
1,350,Agriculture
2,370,Commerce and Housing Credit
3,450,Community and Regional Development
4,500,"Education, Training, Employment, and Social Se..."
5,270,Energy
6,800,General Government
7,250,"General Science, Space, and Technology"
8,000,Governmental Receipts
9,550,Health


## 🏛️ Stage 1A: Collect Budget Functions

**API Endpoint**: `https://api.usaspending.gov/api/v2/budget_functions/list_budget_functions/`

This cell fetches all high-level budget functions (e.g., National Defense, Social Security):
- **Method**: GET request (simple list retrieval)
- **Data Processing**: Uses `pd.json_normalize()` to flatten nested JSON into DataFrame
- **Expected Output**: ~20 budget functions with codes and titles
- **Data Structure**: `budget_function_code`, `budget_function_title`

## 📋 Stage 1B: Collect Budget Subfunctions

**API Endpoint**: `https://api.usaspending.gov/api/v2/budget_functions/list_budget_subfunctions/`

This cell fetches all detailed subfunctions within budget functions:
- **Method**: POST request (may support filtering parameters)
- **Data Processing**: Normalizes JSON response into structured DataFrame
- **Expected Output**: ~70 budget subfunctions with codes and titles
- **Data Structure**: `budget_subfunction_code`, `budget_subfunction_title`

In [ ]:
# 2. Collect all subfunctions
url = "https://api.usaspending.gov/api/v2/budget_functions/list_budget_subfunctions/"
data = requests.post(url).json()
subfunctions = data['results']
df_budget_subfunctions = pd.json_normalize(subfunctions)

In [ ]:
df_budget_subfunctions

,budget_subfunction_code,budget_subfunction_title
0,352,Agricultural research and services
1,402,Air transportation
2,452,Area and regional development
3,053,Atomic energy defense activities
4,803,Central fiscal operations
...,...,...
67,603,Unemployment compensation
68,702,"Veterans education, training, and rehabilitation"
69,704,Veterans housing
70,301,Water resources


## 👀 Data Verification: View Subfunctions

Displays the collected budget subfunctions data for verification:
- **Quality Check**: Ensures data was collected properly
- **Structure Review**: Confirms column names and data types
- **Content Preview**: Shows sample subfunction codes and titles

## 💾 Stage 1 Data Storage

Saves the collected budget functions and subfunctions to Google Drive:

**Output Files**:
- **`budget_functions.csv`**: All budget functions with codes and titles
- **`budget_subfunctions.csv`**: All budget subfunctions with codes and titles

**Storage Benefits**:
- **Persistence**: Data saved permanently to Google Drive
- **Reusability**: Can reload data without re-calling APIs
- **Backup**: Prevents data loss if session ends

In [ ]:
output_folder = '/content/drive/My Drive/USASpendingResults'

# Make sure folder exists
os.makedirs(output_folder, exist_ok=True)

# Save budget functions
functions_file = os.path.join(output_folder, 'budget_functions.csv')
df_budget_functions.to_csv(functions_file, index=False)

# Save budget subfunctions
subfunctions_file = os.path.join(output_folder, 'budget_subfunctions.csv')
df_budget_subfunctions.to_csv(subfunctions_file, index=False)

In [ ]:
#  Load saved CSVs from Drive
functions_file = os.path.join(output_folder, 'budget_functions.csv')
subfunctions_file = os.path.join(output_folder, 'budget_subfunctions.csv')

df_budget_functions = pd.read_csv(functions_file)
df_budget_subfunctions = pd.read_csv(subfunctions_file)


# Ensure codes are integers
#df_budget_functions['budget_function_code'] = df_budget_functions['budget_function_code'].astype(int)
#df_budget_subfunctions['budget_subfunction_code'] = df_budget_subfunctions['budget_subfunction_code'].astype(int)

# Compute next_function_code (upper bound)
df_budget_functions = df_budget_functions.sort_values('budget_function_code').reset_index(drop=True)
df_budget_functions['next_function_code'] = df_budget_functions['budget_function_code'].shift(-1)
df_budget_functions['next_function_code'] = df_budget_functions['next_function_code'].fillna(9999).astype(int)

# Define mapping function
def map_function(sub_code):
    match = df_budget_functions[
        (df_budget_functions['budget_function_code'] <= sub_code) &
        (sub_code < df_budget_functions['next_function_code'])
    ]
    if not match.empty:
        return pd.Series([match.iloc[0]['budget_function_code'], match.iloc[0]['budget_function_title']])
    else:
        return pd.Series([None, None])

# Apply mapping
df_budget_subfunctions[['budget_function_code', 'budget_function_title']] = df_budget_subfunctions['budget_subfunction_code'].apply(map_function)

# Combine and sort
combined_df = df_budget_subfunctions[[
    'budget_function_code', 'budget_function_title',
    'budget_subfunction_code', 'budget_subfunction_title'
]].sort_values(['budget_function_code', 'budget_subfunction_code']).reset_index(drop=True)

# Format codes with leading zeros
for col in ['budget_function_code', 'budget_subfunction_code']:
    combined_df[col] = combined_df[col].astype(int).astype(str).str.zfill(3)


# Save back to Drive
combined_file = os.path.join(output_folder, 'budget_function_subfunction_mapping.csv')
combined_df.to_csv(combined_file, index=False)




## 🔗 Stage 2: Hierarchical Mapping Creation

Creates the critical function-subfunction relationships for Stage 3:

**Mapping Logic**:
- **Range-based Matching**: Uses budget function code ranges to determine which subfunctions belong to each function
- **Next Code Calculation**: Computes upper bounds for each function's range
- **Smart Assignment**: Maps subfunctions to appropriate parent functions based on code proximity

**Output Structure**:
- **`budget_function_code`**: Parent function (e.g., '050' for National Defense)
- **`budget_function_title`**: Function name
- **`budget_subfunction_code`**: Child subfunction (e.g., '051' for Defense-Military)  
- **`budget_subfunction_title`**: Subfunction name

**Code Formatting**: Ensures 3-digit codes with leading zeros for consistency

## 📊 Mapping Verification: View Combined Data

Displays the hierarchical function-subfunction mapping for verification:
- **Hierarchical Structure**: Shows parent-child relationships clearly
- **Data Quality**: Confirms proper code formatting and mapping accuracy  
- **Coverage Check**: Ensures all subfunctions are properly mapped to functions

In [ ]:
combined_df

,budget_function_code,budget_function_title,budget_subfunction_code,budget_subfunction_title
0,000,Governmental Receipts,000,Governmental Receipts
1,050,National Defense,050,National Defense
2,050,National Defense,051,Department of Defense-Military
3,050,National Defense,053,Atomic energy defense activities
4,050,National Defense,054,Defense-related activities
...,...,...,...,...
67,800,General Government,808,Other general government
68,800,General Government,809,Deductions for offsetting receipts
69,900,Net Interest,901,Interest on Treasury debt securities (gross)
70,900,Net Interest,908,Other interest


In [ ]:
output_folder = '/content/drive/My Drive/USASpendingResults'

In [ ]:

result = pd.read_csv('/content/drive/My Drive/USASpendingResults/budget_function_subfunction_mapping.csv')


## 📂 Load Mapping Data for Stage 3

Loads the hierarchical mapping file created in Stage 2:
- **Input File**: `budget_function_subfunction_mapping.csv`
- **Purpose**: Provides the task list for federal account collection
- **Structure**: Each row represents one function-subfunction pair to query

## 🌐 HTTP Session Configuration for Stage 3

Sets up robust HTTP session for extensive federal account data collection:

**Retry Strategy**:
- **Total Retries**: 10 attempts for failed requests
- **Backoff Factor**: 0.5 seconds between retries (exponential backoff)
- **Status Codes**: Retries on server errors (500, 502, 503, 504)
- **Method Support**: Optimized for POST requests

**Why Aggressive Retries**: Stage 3 involves thousands of API calls, so robust error handling is essential

In [ ]:
import requests
from requests.adapters import HTTPAdapter, Retry

def setup_session():
    session = requests.Session()
    retries = Retry(
        total=10,
        backoff_factor=0.5,
        status_forcelist=[500, 502, 503, 504],
        allowed_methods=["POST"]
    )
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('https://', adapter)
    return session


In [ ]:
def fetch_federal_account(session, row, year, quarter):


    function_code = f"{int(row['budget_function_code']):03d}"
    subfunction_code = f"{int(row['budget_subfunction_code']):03d}"

    url = "https://api.usaspending.gov/api/v2/spending/"
    payload = {
        "type": "federal_account",
        "filters": {
            "fy": str(year),
            "quarter": str(quarter),
            "budget_function": function_code,
            "budget_subfunction": subfunction_code
        }
    }

    try:
        resp = session.post(url, json=payload, timeout=30)
        resp.raise_for_status()
        data = resp.json()

        results = data.get('results', [])
        if not results:

            return [], [{
                "budget_function_code": function_code,
                "budget_subfunction_code": subfunction_code,
                "reason": "No results"
            }]

        records = []
        for item in results:
            records.append({
                "fy": year,
                "quarter": quarter,
                "budget_function_code": function_code,
                "budget_subfunction_code": subfunction_code,
                "federal_account_code": item['id'],
                "federal_account_name": item['name'],
                "obligated_amount": item['amount']
            })
        return records, []

    except Exception as e:

        return [], [{
            "budget_function_code": function_code,
            "budget_subfunction_code": subfunction_code,
            "reason": str(e)
        }]


## 💼 Federal Account Data Fetcher

**Core function for collecting detailed federal account information:**

**API Endpoint**: `https://api.usaspending.gov/api/v2/spending/`
**Request Type**: POST with `type="federal_account"`

**Filtering Parameters**:
- **Fiscal Year**: Specific FY (e.g., 2024)
- **Quarter**: Specific quarter (1, 2, 3, or 4)  
- **Budget Function**: 3-digit function code (e.g., '050')
- **Budget Subfunction**: 3-digit subfunction code (e.g., '051')

**Output Data**:
- **`federal_account_code`**: Unique account identifier
- **`federal_account_name`**: Account description
- **`obligated_amount`**: Dollar amount obligated in that quarter

**Error Handling**: Returns both successful records and detailed failure information

## ⚡ Parallel Processing for Quarter Collection

**Orchestrates concurrent API calls for one fiscal year/quarter combination:**

**Parallelization Strategy**:
- **ThreadPoolExecutor**: Manages concurrent worker threads
- **Default Workers**: 10 parallel requests (configurable)
- **Task Distribution**: Each function-subfunction pair becomes one API call

**Input**: DataFrame of function-subfunction mappings from Stage 2
**Processing**: For each row, calls `fetch_federal_account()` in parallel
**Output**: Combined results and failures from all parallel requests

**Performance**: Dramatically reduces collection time from hours to minutes

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_all_for_quarter(session, df, year, quarter, max_workers=10):
    results = []
    failures = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for _, row in df.iterrows():
            futures.append(executor.submit(fetch_federal_account, session, row, year, quarter))

        for future in as_completed(futures):
            res, fail = future.result()
            results.extend(res)
            failures.extend(fail)
    return results, failures


In [ ]:
import os

def save_results(results, failures, year, quarter, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    results_df = pd.DataFrame(results)
    failures_df = pd.DataFrame(failures)

    results_file = f"{output_folder}/federal_accounts_FY{year}_Q{quarter}.csv"
    failures_file = f"{output_folder}/failed_FY{year}_Q{quarter}.csv"

    results_df.to_csv(results_file, index=False)
    failures_df.to_csv(failures_file, index=False)

    print(f"✅ Saved {len(results)} results to {results_file}")
    print(f"⚠️ Saved {len(failures)} failures to {failures_file}")


## 💾 Data Storage per Quarter

**Saves results and failures for each fiscal year/quarter combination:**

**Output Files**:
- **Results**: `federal_accounts_FY{YYYY}_Q{Q}.csv` - Successful data collection
- **Failures**: `failed_FY{YYYY}_Q{Q}.csv` - Failed requests for analysis/retry

**File Organization**:
- **Quarterly Granularity**: Separate files for each quarter enable focused analysis
- **Progress Tracking**: Shows counts of successful vs. failed requests
- **Data Integrity**: Preserves all collected data even if some requests fail

**Benefits**: Enables incremental processing and easy troubleshooting

## 🚀 Stage 3 Master Execution Loop

**Orchestrates the complete federal account data collection across all years and quarters:**

**Date Range Configuration**:
- **Start Year**: 2017 (begins with Q2 due to data availability)
- **End Year**: 2024 
- **Quarter Coverage**: Full 4 quarters for most years, partial for 2017

**Processing Scale**:
- **Total Quarters**: ~30 quarters (7.5 years × 4 quarters)
- **API Calls per Quarter**: ~70 (one per function-subfunction pair)
- **Total API Calls**: ~2,100 requests
- **Expected Runtime**: 2-4 hours depending on API response times

**Progress Tracking**: Displays detailed progress for each fiscal year and quarter

In [ ]:
def run_all_years(df, output_folder, start_year=2017, end_year=2024, max_workers=10):
    session = setup_session()

    for year in range(start_year, end_year + 1):
        if year == 2017:
            quarters = [2, 3, 4]
        else:
            quarters = [1, 2, 3, 4]

        for quarter in quarters:
            print("\n==============================")
            print(f"📌 Fetching FY {year} Q{quarter}")
            print("==============================")

            results, failures = fetch_all_for_quarter(session, df, year, quarter, max_workers=max_workers)
            save_results(results, failures, year, quarter, output_folder)



In [ ]:
run_all_years(result, output_folder, max_workers=10)



📌 Fetching FY 2017 Q2


✅ Saved 1863 results to /content/drive/My Drive/USASpendingResults/federal_accounts_FY2017_Q2.csv
⚠️ Saved 5 failures to /content/drive/My Drive/USASpendingResults/failed_FY2017_Q2.csv

📌 Fetching FY 2017 Q3
✅ Saved 1942 results to /content/drive/My Drive/USASpendingResults/federal_accounts_FY2017_Q3.csv
⚠️ Saved 5 failures to /content/drive/My Drive/USASpendingResults/failed_FY2017_Q3.csv

📌 Fetching FY 2017 Q4


✅ Saved 1967 results to /content/drive/My Drive/USASpendingResults/federal_accounts_FY2017_Q4.csv
⚠️ Saved 5 failures to /content/drive/My Drive/USASpendingResults/failed_FY2017_Q4.csv

📌 Fetching FY 2018 Q1
✅ Saved 1887 results to /content/drive/My Drive/USASpendingResults/federal_accounts_FY2018_Q1.csv
⚠️ Saved 5 failures to /content/drive/My Drive/USASpendingResults/failed_FY2018_Q1.csv

📌 Fetching FY 2018 Q2
✅ Saved 1861 results to /content/drive/My Drive/USASpendingResults/federal_accounts_FY2018_Q2.csv
⚠️ Saved 5 failures to /content/drive/My Drive/USASpendingResults/failed_FY2018_Q2.csv

📌 Fetching FY 2018 Q3
✅ Saved 1942 results to /content/drive/My Drive/USASpendingResults/federal_accounts_FY2018_Q3.csv
⚠️ Saved 5 failures to /content/drive/My Drive/USASpendingResults/failed_FY2018_Q3.csv

📌 Fetching FY 2018 Q4
✅ Saved 1943 results to /content/drive/My Drive/USASpendingResults/federal_accounts_FY2018_Q4.csv
⚠️ Saved 5 failures to /content/drive/My Drive/USASpendingResults/fail

✅ Saved 1909 results to /content/drive/My Drive/USASpendingResults/federal_accounts_FY2024_Q2.csv
⚠️ Saved 4 failures to /content/drive/My Drive/USASpendingResults/failed_FY2024_Q2.csv

📌 Fetching FY 2024 Q3
✅ Saved 1919 results to /content/drive/My Drive/USASpendingResults/federal_accounts_FY2024_Q3.csv
⚠️ Saved 4 failures to /content/drive/My Drive/USASpendingResults/failed_FY2024_Q3.csv

📌 Fetching FY 2024 Q4
✅ Saved 1967 results to /content/drive/My Drive/USASpendingResults/federal_accounts_FY2024_Q4.csv
⚠️ Saved 4 failures to /content/drive/My Drive/USASpendingResults/failed_FY2024_Q4.csv


## ▶️ Execute Complete Data Collection

**This cell starts the full federal account data collection process:**

**What Happens**:
1. **Session Initialization**: Sets up HTTP session with retry logic
2. **Year-by-Year Processing**: Iterates through FY 2017-2024
3. **Quarter-by-Quarter Collection**: Processes all 4 quarters per year
4. **Parallel API Calls**: 10 concurrent requests per quarter
5. **Data Storage**: Saves results and failures for each quarter

**Expected Output**: Detailed progress reports and file creation confirmations

**Runtime**: 2-4 hours for complete collection (~2,100 API calls)

## 🔍 Post-Collection Analysis: Failure Code Union

**Analyzes and consolidates all failed requests across the entire collection:**

**Purpose**:
- **Quality Assessment**: Identifies which function-subfunction pairs consistently fail
- **Data Gaps**: Reveals missing coverage in the federal account data
- **Retry Strategy**: Provides clean list of codes to retry or investigate

**Process**:
1. **File Discovery**: Scans output folder for all failure CSV files
2. **Data Consolidation**: Combines failures from all quarters into single dataset
3. **Deduplication**: Removes duplicate function-subfunction pairs
4. **Union Generation**: Creates master list of all unique failed codes

**Output**: Clean CSV file with all unique failed function-subfunction combinations

In [ ]:
import pandas as pd
import os

def generate_union_of_failed_codes(input_folder, output_file):
    all_failures = []

    # ✅ Loop through all CSV files in the folder
    for fname in os.listdir(input_folder):
        if fname.endswith(".csv") and "failed" in fname.lower():
            path = os.path.join(input_folder, fname)
            print(f"📌 Reading: {fname}")
            try:
                df = pd.read_csv(path)
                all_failures.append(df)
            except Exception as e:
                print(f"⚠️ Could not read {fname}: {e}")

    # ✅ Combine all into one DataFrame
    if not all_failures:
        print("⚠️ No failure files found!")
        return

    combined = pd.concat(all_failures, ignore_index=True)

    # ✅ Keep only function and subfunction codes
    combined = combined[['budget_function_code', 'budget_subfunction_code']]

    # ✅ Drop duplicates
    combined = combined.drop_duplicates().sort_values(['budget_function_code', 'budget_subfunction_code'])

    print(f"✅ Total unique failure codes: {len(combined)}")

    # ✅ Save result
    combined.to_csv(output_file, index=False)
    print(f"✅ Union of failed codes saved to: {output_file}")

    return combined


In [ ]:
input_folder = '/content/drive/My Drive/USASpendingResults'
output_file = '/content/drive/My Drive/USASpendingResults/union_failed_codes.csv'

union_df = generate_union_of_failed_codes(input_folder, output_file)


📌 Reading: failed_FY2018_Q1.csv
📌 Reading: failed_FY2017_Q2.csv
📌 Reading: failed_FY2017_Q3.csv
📌 Reading: failed_FY2017_Q4.csv
📌 Reading: failed_FY2018_Q2.csv
📌 Reading: failed_FY2018_Q3.csv
📌 Reading: failed_FY2019_Q1.csv
📌 Reading: failed_FY2018_Q4.csv
📌 Reading: failed_FY2019_Q2.csv
📌 Reading: failed_FY2019_Q3.csv
📌 Reading: failed_FY2019_Q4.csv
📌 Reading: failed_FY2020_Q1.csv
📌 Reading: failed_FY2020_Q2.csv
📌 Reading: failed_FY2020_Q3.csv
📌 Reading: failed_FY2020_Q4.csv
📌 Reading: failed_FY2021_Q1.csv
📌 Reading: failed_FY2021_Q2.csv
📌 Reading: failed_FY2021_Q3.csv
📌 Reading: failed_FY2021_Q4.csv
📌 Reading: failed_FY2022_Q1.csv
📌 Reading: failed_FY2022_Q2.csv
📌 Reading: failed_FY2022_Q3.csv
📌 Reading: failed_FY2022_Q4.csv
📌 Reading: failed_FY2023_Q1.csv
📌 Reading: failed_FY2023_Q2.csv
📌 Reading: failed_FY2023_Q3.csv
📌 Reading: failed_FY2023_Q4.csv
📌 Reading: failed_FY2024_Q3.csv
📌 Reading: failed_FY2024_Q4.csv
📌 Reading: failed_FY2024_Q1.csv
📌 Reading: failed_FY2024_Q2.csv
📌 Readin

## 📊 Execute Failure Analysis

**Runs the failure consolidation process:**

**Configuration**:
- **Input Folder**: `/content/drive/My Drive/USASpendingResults` (all quarterly failure files)
- **Output File**: `union_failed_codes.csv` (consolidated failure list)

**Expected Results**:
- **File Count**: Number of failure files processed
- **Unique Failures**: Count of distinct function-subfunction pairs that failed
- **Success Rate**: Insight into overall data collection effectiveness

**Use Case**: This data helps identify patterns in API limitations or data availability gaps

## 👀 Final Verification: View Failed Codes

**Displays the consolidated failure analysis results:**

**Important Understanding**: 
- **Failed codes ≠ API errors** - These represent function-subfunction pairs with **no federal account data available** on USASpending.gov for the requested years/quarters
- **Data Availability Gaps** - Some budget categories simply don't have federal account activity in certain time periods
- **Website Limitations** - The absence of data reflects actual government spending patterns, not technical failures

**Data Review**:
- **No Data Combinations**: Shows which function-subfunction pairs had no federal accounts in specific periods
- **Spending Patterns**: Reveals which budget categories are inactive or have limited activity
- **Data Completeness**: Confirms successful API calls that returned empty results (not errors)

**Next Steps**: This information helps with:
- **Analysis Planning** - Understanding which categories have consistent data vs. sporadic activity  
- **Data Quality Assessment** - Distinguishing between technical failures and legitimate data absence
- **Research Focus** - Identifying budget areas with reliable historical data for trend analysis

In [ ]:
union_df

,budget_function_code,budget_subfunction_code
140,0,0
0,50,50
142,50,51
143,50,53
144,50,54
145,150,152
1,370,372
2,400,400
148,600,604
149,600,605
